In [5]:
import numpy as np
import sys
import os
sys.path.append("../")
# Packages for images
from PIL import Image, ImageOps

from src.compress_sensing_library import *
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
img_nm = 'city_part2'
root = search_root()
load_V1 = '{root}/result/dct/{img}/V1/param_Tue_May_9_21_38_35_2023.csv'.format(root = root, img = img_nm)
load_pixel = '{root}/result/dct/{img}/pixel/param_Tue_May_9_21_08_59_2023.csv'.format(root = root, img = img_nm)
load_gaussian = '{root}/result/dct/{img}/gaussian/param_Tue_May_9_21_10_24_2023.csv'.format(root = root, img = img_nm)
directory = "num_cell_error"
save = True

In [7]:
V1_param_df = pd.read_csv(load_V1)

for index in V1_param_df.columns:
    if (index == 'Unnamed: 0') :
        V1_param_df.drop('Unnamed: 0', axis = 1, inplace=True)

In [16]:
V1_param_mean_df = V1_param_df.groupby(
    ['num_cell', 'sparse_freq', 'cell_size', 'alp'], as_index=False).mean().drop('rep', axis=1) 

V1_param_min_df = V1_param_mean_df.sort_values('error').drop_duplicates('num_cell')
V1_param_min_df = V1_param_min_df.rename(columns={'error': 'min_error'})
V1_merged_df = pd.merge(V1_param_df, V1_param_min_df, 
                               on=['num_cell', 'sparse_freq', 'cell_size', 'alp'], how='left')
V1_plotting_data = V1_merged_df.loc[V1_merged_df['min_error'].notnull()]

In [14]:
#V1 Data processing
V1_min_mean_err_df = pd.DataFrame()
for i in V1_param_mean_df['num_cell'].unique():
    V1_temp = V1_param_mean_df.loc[V1_param_mean_df['num_cell'] == i]
    #hyperparameter for each number of cell
    ## Grabbing values by each values
    V1_min_mean_err_df = V1_min_mean_err_df.append(V1_temp.loc[V1_temp['error'] == V1_temp['error'].min()])
    
# Merge two data to extract
V1_min_mean_err_df = V1_min_mean_err_df.rename(columns={'error' : 'mean_err'})
V1_merged_df = pd.merge(V1_param_df, V1_min_mean_err_df, on = ['num_cell', 'sparse_freq', 'cell_size', 'alp'], how = 'left')
V1_plotting_data = V1_merged_df.loc[V1_merged_df['mean_err'].notnull()]


In [17]:
V1_plotting_data

,rep,alp,num_cell,cell_size,sparse_freq,error,min_error
246,0,0.01,500,8,2,8.154232,8.239167
341,0,0.10,300,6,2,10.776919,11.127548
412,0,1.00,100,4,4,16.927424,17.231876
441,0,1.00,200,6,2,13.992448,14.022165
517,0,10.00,50,6,4,19.183644,19.857467
...,...,...,...,...,...,...,...
16871,19,0.01,500,8,2,8.322520,8.239167
16966,19,0.10,300,6,2,11.162671,11.127548
17037,19,1.00,100,4,4,17.491429,17.231876
17066,19,1.00,200,6,2,14.038472,14.022165
